In [1]:
# reshape only can be done on numpy array can we use tf's built-in data plz
# want to make this simplest it can be at first. how simple can 2x2 be??

In [2]:
# assignment 6
# MSDS 422
# Professor Anil Chaturvedi

# https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
# https://github.com/ageron/handson-ml/blob/master/12_distributed_tensorflow.ipynb

# ??
# https://www.datacamp.com/community/tutorials/cnn-tensorflow-python

<div style="text-align: right"><b>pkg imports</b></div>

In [3]:
# basix
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import time

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#modeling
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import tensorflow as tf

# other

%matplotlib inline   
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<div style="text-align: right"><b>configs, table setup</b></div>

In [4]:
pd.options.display.float_format = "{:,.4f}".format
RANDOM_SEED = 8675309
responseVar = "label"

In [5]:
# setup for consolidated table, later

table = {"model 1":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 2":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 3":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 4":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
        }

pd.DataFrame(table).transpose()

,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,None,None,None,None,None
model 2,None,None,None,None,None
model 3,None,None,None,None,None
model 4,None,None,None,None,None


<div style="text-align: right"><b>defining things for model</b></div>

In [6]:
# activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# lambda x: 1 / (1 + np.exp(-x))

In [7]:
training_iters = 10
learning_rate = 0.01
b_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

<div style="text-align: right"><b>data setup</b></div>

In [8]:
mnist_train = pd.read_csv('train.csv')
mnist_test = pd.read_csv('test.csv')
#mnist = pd.concat([mnist_test, mnist_train.drop(columns = "label")]) # combined dataset

In [9]:
#mnist_archive = mnist
mnist_train_archive = mnist_train
mnist_test_archive = mnist_test

In [10]:
pd.DataFrame(mnist_train.describe()["pixel774"])

,pixel774
count,"42,000.0000"
mean,0.2193
std,6.3129
min,0.0000
25%,0.0000
50%,0.0000
75%,0.0000
max,254.0000


In [11]:
scaler = StandardScaler()
np_mnist_train = scaler.fit_transform(mnist_train)
mnist_train = pd.DataFrame(np_mnist_train, index=mnist_train.index, columns=mnist_train.columns)

np_mnist_test = scaler.fit_transform(mnist_test)
mnist_test = pd.DataFrame(np_mnist_test, index=mnist_test.index, columns=mnist_test.columns)
pd.DataFrame(mnist_test.describe()["pixel774"])

,pixel774
count,"28,000.0000"
mean,-0.0000
std,1.0000
min,-0.0301
25%,-0.0301
50%,-0.0301
75%,-0.0301
max,46.1952


In [12]:
true_X_train = mnist_train.loc[:, mnist_train.columns != responseVar]
true_y_train = mnist_train[responseVar]
true_X_test = mnist_test.loc[:, mnist_test.columns != responseVar]
#y_test does not exist

In [13]:
X = true_X_train
y = true_y_train

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED)

<div style="text-align: right"><b>model 2 : 2x2. 2 layers w 2 neurons each</b></div>

In [15]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter()

approximate start time :  Tue Oct 20 00:19:47 2020


In [16]:
twox1 = MLPClassifier(hidden_layer_sizes = (2, 2), 
                      activation='logistic', 
                      alpha = learning_rate,
                      batch_size = b_size).fit(X_train, y_train)

ValueError: Unknown label type: (array([-0.50443086,  0.88075661, -0.50443086, ...,  1.57335035,
        0.53445975,  1.22705348]),)

In [ ]:
#twox2.fit(X_train, y_train)
#probs = twox2.predict_proba(X_test[:1])
#preds = twox2.predict(X_test[:5, :])

In [ ]:
stop2 = time.perf_counter()
tm2 = round(stop2 - start2)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm2)
print("time of execution in minutes : ", round(tm2/60, 1))

In [ ]:
table["model 2"]["nbr_layers"] = "2"
table["model 2"]["nodes_p_layer"] = "2"
table["model 2"]["processing_time"] = tm2
table["model 2"]["testset_acc"] = twox2.score(X_test, y_test)
table["model 2"]["trainset_acc"] = twox2.score(X_train, y_train)

In [ ]:
# for tf; X and y need be np.array, not pd.DF
#train_X = X.reshape(-1, 28, 28, 1)
#test_X = y.reshape(-1, 28, 28, 1)

In [ ]:
print("processing time in seconds : ")
pd.DataFrame(table).transpose()